In [14]:
!pip install praw
!pip install pandas numpy scikit-learn
!pip install matplotlib scipy PyDrive
!pip install gensim nltk

     |████████████████████████████████| 12.4 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 987 kB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 16.5 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 8.2 MB/s eta 0:00:011
     |████████████████████████████████| 58 kB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 14.3 MB/s eta 0:00:01
     |████████████████████████████████| 89 kB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 3.6 MB/s eta 0:00:01
  Created wheel for PyDrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27435 sha256=c3540fe85da1074417fc439806894b70cfe88ce4786348265a6daafc7d59bb5e
  Stored in directory: /tmp/pip-ephem-wheel-cache-uscymxx_/wheels/dd/8f/dd/be434b75e07ab06eaa16cbaf7d3e03d8ad68c12929ec3022a4
  Created wheel for googleapis-common-protos: filename=googleapis_common_protos-1.51.0-py3-none-any.whl size=

     |████████████████████████████████| 24.2 MB 5.5 MB/s eta 0:00:011
     |████████████████████████████████| 105 kB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 28.8 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 4.7 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.11.1-py3-none-any.whl size=95256 sha256=851961d592174f45deda572e33aaf009f36fd0e1f9ec0b0aa99d04f8c608be95
  Stored in directory: /tmp/pip-ephem-wheel-cache-kwvf7qlo/wheels/89/6b/f0/391fb74c038a4b87d4ac6ef4cd4c96ea820ec39b60f78d03a1
Successfully built smart-open


In [17]:
import pymongo
from pymongo import MongoClient
import numpy as np
import praw
from praw.models import MoreComments
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

In [18]:
client = MongoClient('mongodb://localhost:27017/')
db = client["reddit"]

In [19]:
reddit = praw.Reddit(client_id = "F4OMSR9jAi_kFQ",
                     client_secret = "dBKDarfWBg7QyeZ-CXlNeLYlQ9A",
                     user_agent = "web_app:reddit_scraper:v1.1.1 (by u/skadwhoosh)",
                     username = "skadwhoosh",
                     password = "reddit@123")

flairs = ["AskIndia", "Non-Political", "[R]eddiquette", 
          "Scheduled", "Photography", "Science/Technology",
          "Politics", "Business/Finance", "Policy/Economy",
          "Sports", "Food", "AMA"]

subreddit = reddit.subreddit('india')

In [20]:
for flair in flairs:

    #The posts' data is collected by searching by the flair name in the list. Top 100 posts are recorded and stored for analysis.

    relevant_subreddits = subreddit.search(f"flair_name:{flair}",limit=100)

    for submission in relevant_subreddits:
        posts = {
        "title":str(submission.title),
        "score":str(submission.score),
        "id":str(submission.id),
        "url":str(submission.url),
        "comms_num":str(submission.num_comments),
        "created":str(submission.created),
        "body":str(submission.selftext),
        "author":str(submission.author),
        "flair":str(flair),	
        }

#  Only top ten comments and their authors are considered. 
        submission.comments.replace_more(limit=None)
        comment = ''
        authors = ''
        count = 0
        for top_level_comment in submission.comments:
            comment = comment + ' ' + top_level_comment.body
            authors = authors + ' ' + str(top_level_comment.author)
            count+=1     
            if(count > 10):
                break

        posts["comment"] = str(comment)
        posts["authors"] = str(authors)
        result = db.posts.insert_one(posts)

In [24]:
replace_by_space = re.compile('[/(){}\[\]\|@,;]')
replace_symbol = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = replace_by_space.sub(' ', text) # replace certain symbols by space in text
    text = replace_symbol.sub('', text) # delete symbols from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove STOPWORDS from text
    return text

In [35]:
posts['title']

0      Lost my Job, Sick Mother and Paralysed Dad, In...
1      Why does the government come with a begging bo...
2      Mother's condition is going worse due to hepat...
3      People stuck with their family during the lock...
4      Men who are 30+ and have decided not to get ma...
5      [Please Advice] Reality punched me in the face...
6      What is the PM CARES fund being used for? how ...
7      r/India: If money is no bar, would you prefer ...
8      So, I'm an American dating a South Indian, and...
9                     What is absolutely worth the money
10                   Any idea what has happened to quora
11     She is leaving me because of me being poor wha...
12                             Got struck by a cop today
13     People of r/india, how is your Work from Home ...
14     How much yelling is normal for Indians? Is it ...
15                                   Urgent, please help
16     Skincare India, what are your holy grail produ...
17     Did Ramayana actually ha

In [2]:
collection = db.posts
posts = pd.DataFrame(list(collection.find()))

del posts['_id']

# posts['title'] = posts['title'].apply(str())
# posts['body'] = posts['body'].apply(str())
# posts['comment'] = posts['comment'].apply(str())

posts['title'] = posts['title'].apply(clean_text)
posts['body'] = posts['body'].apply(clean_text)
posts['comment'] = posts['comment'].apply(clean_text)

combined_features = posts["title"] + posts["comment"] + posts["url"] + posts["body"]
posts = posts.assign(combined_features = combined_features)

#saving the csv file
posts.to_csv('../Data/data.csv',index=False)
posts.to_csv('data.csv',index=False)